**TODOs**
- [ ] The trained SciBERT model `scibert_chemprot.tar.gz` stores inside itself 
  absolute paths to vocabulary text and weights! So it cannot be move around
  without rewriting its metadata inside.
- [ ] SciBERT can not be obtained with `pip install`, so currently one needs to 
    1. `git clone https://github.com/allenai/scibert.git`
    2. `export PYTHONPATH=$PYTHONPATH:PATH_TO_SCIBERT`


# Goal of the notebook
(to be completed)

In [ ]:
from collections import OrderedDict
import logging
from pathlib import Path
import sqlite3

import IPython
import ipywidgets
import numpy as np
import pandas as pd
import scispacy
import spacy
import nltk

import bbsearch.embedding_models as embedding_models
from bbsearch.mining import ChemProt, TextMiningPipeline
from bbsearch.widget import Widget
from bbsearch.article_saver import ArticleSaver

# Set a Project
The User choses/creates a project to host a KG.

# Set topic
The user defines its topic.

# Data Import
The user loads data from a data source (CORD-19).
The loaded data forms the corpus.
The user searches the CORPUS in Blue Brain Search.

In [ ]:
DATASET_VERSION = 'v7'

data_path = Path("/raid/covid_data/data/") / DATASET_VERSION
assets_path = Path("/raid/covid_data/assets")
embeddings_path = data_path / "embeddings"

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

models_to_load = ["SBioBERT","BSV"]
embeddings_models = dict()
precomputed_embeddings = dict()
for model in models_to_load:
    if model == 'BSV':
        embeddings_models[model] = embedding_models.BSV(checkpoint_model_path=assets_path / 'BioSentVec_PubMed_MIMICIII-bigram_d700.bin')
    else:
        embeddings_models[model] = getattr(embedding_models, model)()
        
    precomputed_embeddings[model] = np.load(embeddings_path / model / f'{model}.npy').astype('float32')  
    # astype('float32') speeds up the search
database = sqlite3.connect(data_path / 'cord19_v1.db')

In [ ]:
article_saver = ArticleSaver(database=database)

In [ ]:
bbs_widget = Widget(embeddings_models, precomputed_embeddings, database, article_saver=article_saver)
bbs_widget.display()

Status of the Article Saver

In [ ]:
table = article_saver.summary_table()
display(table)

# Set schemas
The user defines the KG schema.

# Create a knowledge graph according to schemas
The user extracts data from the text of a set of papers using selected Named Entity Recognizers and Relation Extractors from Blue Brain Search.
The user can preview the extracted data.
The user curates extracted data.
The user links the extracted entities and relations to ontologies.
The user saves data into Knowledge Graph.

- **input**: raw text
- **output**: csv table of extracted entities/relations

In [ ]:
DEFAULT_TEXT = """Autophagy maintains tumour growth through circulating
arginine. Autophagy captures intracellular components and delivers them to
lysosomes, where they are degraded and recycled to sustain metabolism and to
enable survival during starvation. Acute, whole-body deletion of the essential 
autophagy gene Atg7 in adult mice causes a systemic metabolic defect that 
manifests as starvation intolerance and gradual loss of white adipose tissue, 
liver glycogen and muscle mass.  Cancer cells also benefit from autophagy. 
Deletion of essential autophagy genes impairs the metabolism, proliferation, 
survival and malignancy of spontaneous tumours in models of autochthonous 
cancer. Acute, systemic deletion of Atg7 or acute, systemic expression of a 
dominant-negative ATG4b in mice induces greater regression of KRAS-driven 
cancers than does tumour-specific autophagy deletion, which suggests that host 
autophagy promotes tumour growth.
""".replace('\n', ' ').replace('  ', ' ')

In [ ]:
# Entities Extractors (EE)
ee_model = spacy.load("en_ner_craft_md")

# Relations Extractors (RE)
PATH_CHEMPROT_TRAINED_MODEL = assets_path / 'scibert_chemprot.tar.gz'
re_models = {('CHEBI', 'GGP'): [ChemProt(PATH_CHEMPROT_TRAINED_MODEL)]}

# Full Pipeline
text_mining_pipeline = TextMiningPipeline(ee_model, re_models)

In [ ]:
# This is the output: csv table of extracted entities/relations.
table_extractions = None

In [ ]:
# Define Widgets
bbs_widgets = OrderedDict()

In [ ]:
bbs_widgets['articles_button'] = ipywidgets.Button(
    description='Mine Selected Articles!',
    layout=ipywidgets.Layout(width='60%')
)

def article_button(b):
    global table_extractions
    bbs_widgets['out'].clear_output()
    complete_text = ''
    with bbs_widgets['out']:
        article_saver.retrieve_text()
        table_extractions = pd.DataFrame()
        for article_infos, text in article_saver.articles_text.items():
            table_extractions = table_extractions.append(text_mining_pipeline(text, 
                                                                              article_id=article_infos[0], 
                                                                              return_prob=True))
        display(table_extractions)
        
bbs_widgets['articles_button'].on_click(article_button)

In [ ]:
# "Input Text" Widget
bbs_widgets['input_text'] = ipywidgets.Textarea(
        value=DEFAULT_TEXT,
        layout=ipywidgets.Layout(width='75%', height='300px')
    )

# "Submit!" Button
bbs_widgets['submit_button'] = ipywidgets.Button(
    description='Mine This Text!',
    layout=ipywidgets.Layout(width='30%')
)
def cb(b):
    global table_extractions
    bbs_widgets['out'].clear_output()
    with bbs_widgets['out']:
        text = bbs_widgets['input_text'].value
        table_extractions = text_mining_pipeline(text, return_prob=True)
        display(table_extractions)
bbs_widgets['submit_button'].on_click(cb)

# "Output Area" Widget
bbs_widgets['out'] = ipywidgets.Output(layout={'border': '0.5px solid black'})

# Finalize Widgets
ordered_widgets = list(bbs_widgets.values())
main_widget = ipywidgets.VBox(ordered_widgets)
IPython.display.display(main_widget)

- **input**: csv table of extracted entities/relations
- **output**: knowledge graph

In [ ]:
# TEMPORARY Enable people to run the following part outside of DGX-1.
import pandas as pd

! wget -O extractions_example.csv 'https://drive.google.com/uc?export=download&id=11BBtkKsamru4kjUNev8lO_ulNMf7m3Ta'

table_extractions = pd.read_csv('extractions_example.csv', sep=';')

In [ ]:
print(f'The table has {table_extractions.shape[0]} rows.')

In [ ]:
from typing import Iterator, Dict
import pandas as pd

def represent_as_annotations(df: pd.DataFrame) -> Iterator[Dict]:
    def _(row):
        return {
            '@context': 'http://www.w3.org/ns/anno.jsonld',
            '@id': f'https://bbp.epfl.ch/covid19/{row.Index}',
            '@type': 'Annotation',
            'target': {
                'source': row.paper_id,
                'selector': {
                    '@type': 'TextPositionSelector',
                    'start': row.start,
                    'end': row.end,
                    'value': row.entity,
                },
            },
        }
    return (_(x) for x in df.itertuples())

In [ ]:
annotations = list(represent_as_annotations(table_extractions))

In [ ]:
import faiss
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

class Candidate:
    
    def __init__(self, identifier, term, score):
        self.identifier = identifier
        self.term = term
        self.score = score
    
    def __repr__(self):
        attrs = (f"{k}={v!r}" for k, v in self.__dict__.items())
        return f"Candidate({', '.join(attrs)})"

class EntityLinker:
    
    def __init__(self, ontology):
        self.terms = None
        self.model = None
        self.index = None
        self.ontology = list(ontology.items())
    
    def link(self, mentions):
        # TODO Entity disambiguation is coming as the next step.
        return self.candidates(mentions, limit=1)
    
    def disambiguate(self):
        # TODO Entity disambiguation is coming as the next step.
        pass
    
    def candidates(self, mentions, limit=3, threshold=0.6):
        embeddings = self.model.transform(mentions)
        distances, indexes = self.index.search(embeddings.toarray(), limit)
        return [[Candidate(*self.terms[i], d) for i, d in zip(indexes[k], distances[k])
                 if d <= threshold] for k in range(len(mentions))]
    
    def train(self):
        self.model = TfidfVectorizer(analyzer='char', ngram_range=(3, 3), max_df=0.95,
                                     max_features=int(len(self.ontology)*0.1),
                                     dtype=np.float32, norm='l2')
        terms = [x for _, x in self.ontology]
        embeddings = self.model.fit_transform(terms)
        flags = np.array(embeddings.sum(axis=1) != 0).reshape(-1)
        filtered_embeddings = embeddings[flags]
        self.terms = [term for term, flag in zip(self.ontology, flags) if flag]
        self.index = faiss.IndexFlatL2(filtered_embeddings.shape[1])
        self.index.add(filtered_embeddings.toarray())

In [ ]:
# TEMPORARY Retrieve entity_linking-terms.pkl (~ 5 MB).
! wget -O entity_linking-terms.pkl 'https://drive.google.com/uc?export=download&id=1DyA9WL1YpEBO37KkDSCY3f1LWFfscO7F'

In [ ]:
import pickle

with open('entity_linking-terms.pkl', 'rb') as f:
    # TODO Linking to more ontologies than ChEBI is coming in the next steps.
    ontology = pickle.load(f)

linker = EntityLinker(ontology)

linker.train()

# Note: Takes around 12 secs on a BBP issued MacBook Pro.

In [ ]:
from typing import Iterable, Dict, Iterator
from copy import deepcopy

def enrich_annotations(annotations: Iterable[Dict], linker: EntityLinker) -> Iterator[Dict]:
    def _(ann, can):
        new = deepcopy(ann)
        if can:
            # TODO Entity disambiguation is coming as the next step.
            new['body'] = {
                '@id': can[0].identifier,
                'label': can[0].term,
            }
        return new
    mentions = [x['target']['selector']['value'] for x in annotations]
    linked_mentions = linker.link(mentions)
    return (_(ann, can) for ann, can in zip(annotations, linked_mentions))

In [ ]:
enriched_annotations = list(enrich_annotations(annotations, linker))

In [ ]:
import json
from typing import Iterable, Dict
from rdflib import Graph

def load_knowledge_graph(jsonlds: Iterable[Dict]) -> Graph:
    g = Graph()
    for x in jsonlds:
        g.parse(data=json.dumps(x), format='json-ld')
    return g

In [ ]:
knowledge_graph = load_knowledge_graph(enriched_annotations)

In [ ]:
print(f'The knowledge graph has {len(knowledge_graph)} triples.')

# Validate the knowledge graph
Thee User reviews content of Knowledge Graph.

# Correct knowledge graph
The correct the Knowledge Graph is errors occur.

# Access the knowledge graph
The user can search, visualize, and export the knowledge graph.

# Version the knowledge graph
The user can save a knowledge graph with a version.